In [1352]:
import torch

import torch.nn as nn

import torch.nn.functional as F

import torchvision

import torchvision.transforms as transforms

import matplotlib.pyplot as plt

import numpy as np

import math

import time

In [1353]:
torch.cuda.is_available()

True

In [1354]:
%run Data_Processing.ipynb

Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 42.79it/s]


The proportion of 0's in the genomic dataset is 33.34 %
The proportion of 1's in the genomic dataset is 33.34 %
The proportion of 2's in the genomic dataset is 33.32 %
The proportion of NaN's in the genomic dataset is 0.00 %
The proportion of having gout is 0.41


Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 49.77it/s]
C:\Users\tuanh\AppData\Local\Temp\ipykernel_7648\2141829978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['encoded_sex'] = np.select([df['sex'] == "Male" , df['sex'] == "Female"], [1, 2])
C:\Users\tuanh\AppData\Local\Temp\ipykernel_7648\2141829978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['encoded_sex'] = np.select([df['sex'] == "Male" , df['sex'] == "Female"], [1, 2]

0.6449494949494949
              precision    recall  f1-score   support

       False     0.6969    0.7035    0.7002      1167
        True     0.5686    0.5609    0.5647       813

    accuracy                         0.6449      1980
   macro avg     0.6328    0.6322    0.6325      1980
weighted avg     0.6442    0.6449    0.6446      1980



C:\Users\tuanh\anaconda3\envs\pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1355]:
tr_snv.shape

torch.Size([9899, 2000])

In [1356]:
tr_phenos.shape

torch.Size([9899, 5])

## Attention Scoring (Scaled Dot Product)

In [1357]:
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self, dropout):
        
        super().__init__()
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, queries, keys, values):
        
        weights = torch.bmm(queries, torch.transpose(keys, 1, 2)) / math.sqrt(queries.shape[-1])
        
        self.attn_weights = F.softmax(weights, dim = -1)
        
        scores = torch.bmm(self.dropout(self.attn_weights), values)
        
        return scores    

## Normalization

In [1358]:
class Normalize(nn.Module):
    
    def __init__(self, norm_shape):
        
        super().__init__()
        
        self.norm = nn.LayerNorm(norm_shape)
        
    def forward(self, inputs):
        
        return self.norm(inputs)

## Residual Connection + Normalization

In [1359]:
class AddNorm(nn.Module):
    
    def __init__(self, norm_shape):
        
        super().__init__()
        
        self.norm = nn.LayerNorm(norm_shape)
        
    def forward(self, inputs, outputs):
        
        return self.norm(inputs + outputs)

## Initialize Latent Array

In [1360]:
class LatentArray(nn.Module):
    
    def __init__(self, latent_size, latent_dim):
        
        super().__init__()
        
        self.latent_size = latent_size
        
        self.latent_dim = latent_dim
        
    def forward(self, inputs):
        
        latent_array = nn.Parameter(torch.rand(self.latent_size, self.latent_dim))
        
        torch.nn.init.trunc_normal_(latent_array, mean = 0.0, std = 0.02, a = -2.0, b = 2.0)
        
        batch_latent_array = latent_array.repeat(inputs.shape[0], 1, 1)
        
        return batch_latent_array   

## Dense Multi-layer Perceptron Block

In [1361]:
class DenseBlock(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, bias = False):
        
        super().__init__()
        
        self.norm = nn.LayerNorm(input_dim)
        
        self.layer1 = nn.Linear(input_dim, hidden_dim, bias = bias)
        
        self.layer2 = nn.Linear(hidden_dim, input_dim, bias = bias)
        
    def forward(self, inputs):
        
        # Layer Normalize the inputs
        
        outputs = self.norm(inputs)
        
        # Pass through the first linear layer & activate with GELU
        
        first_outputs = F.gelu(self.layer1(outputs))
        
        # Pass through the final linear layer
        
        final_outputs = self.layer2(first_outputs)
        
        return final_outputs

## Fully Connected FeedForward Neural Network Layer

In [1362]:
class FFNLayer(nn.Module):
    
    def __init__(self, embedded_dim, hidden_size, output_size):
        
        super().__init__()
        
        self.layer1 = nn.Linear(embedded_dim, hidden_size)
        
        self.layer2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, inputs):
        
        first_outputs = F.relu(self.layer1(inputs))
        
        final_outputs = self.layer2(first_outputs)
        
        return final_outputs

## Multi Heads Attention (for Self-Attention)

In [1363]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, latent_dim, hidden_dim, num_heads, dropout, bias = False):
        
        super().__init__()
        
        self.num_heads = num_heads
        
        self.attention = ScaledDotProductAttention(dropout)
        
        self.norm = Normalize(latent_dim)
        
        self.addnorm = AddNorm(latent_dim)
        
        self.queries_weights = nn.Linear(latent_dim, latent_dim, bias = bias)
        
        self.keys_weights = nn.Linear(latent_dim, latent_dim, bias = bias)
        
        self.values_weights = nn.Linear(latent_dim, latent_dim, bias = bias)
        
        # self.heads_weights = nn.Linear(latent_dim, latent_dim, bias = bias)
        
        self.dense = DenseBlock(latent_dim, hidden_dim)
        
    def reshape_multi_heads(self, inputs):
        
        # batch size, number of queries/keys/values, number of heads, dimensions / number of heads
        
        inputs = inputs.reshape(inputs.shape[0], inputs.shape[1], self.num_heads, -1)
        
        # batch size, number of heads, number of queries/keys/values, dimensions / number of heads
        
        inputs = inputs.permute(0, 2, 1, 3)
        
        # batch size x number of heads, number of queries/keys/values, dimensions / number of heads

        inputs = inputs.reshape(-1, inputs.shape[2], inputs.shape[3])
        
        return inputs
    
    def reshape_output(self, outputs):
        
        # batch size, number of heads, number of queries/keys/values, dimensions / number of heads
        
        outputs = outputs.reshape(-1, self.num_heads, outputs.shape[1], outputs.shape[2])
        
        # batch size, number of queries/keys/values, number of heads, dimensions / number of heads

        outputs = outputs.permute(0, 2, 1, 3)
        
        # batch size, number of queries/keys/values, dimensions
        
        outputs = outputs.reshape(outputs.shape[0], outputs.shape[1], -1)
        
        return outputs
        
    def forward(self, queries, keys, values):
        
        # Layer Normalize inputs
        
        norm_queries = self.norm(queries)
        
        norm_keys = self.norm(keys)
        
        norm_values = self.norm(values)
        
        # Re-shape QKV into multiple heads and attach learnable parameters
        
        new_queries = self.reshape_multi_heads(self.queries_weights(norm_queries))
        
        new_keys = self.reshape_multi_heads(self.keys_weights(norm_keys))
        
        new_values = self.reshape_multi_heads(self.values_weights(norm_values))
        
        # Perform attention scoring method
        
        outputs = self.attention(new_queries, new_keys, new_values)
        
        # Re-shape the outputs into their original shape (same with keys & values)

        outputs = self.reshape_output(outputs)
        
        # Attach learnable parameters to heads
        
        #outputs = self.heads_weights(outputs)
        
        # Residual Connection & Normalization
        
        outputs = self.addnorm(queries, outputs)
        
        # Pass through Dense Block
        
        #final_outputs = self.dense(outputs)
        
        final_outputs = outputs
        
        return final_outputs

## Cross Attention

In [1364]:
class CrossAttention(nn.Module):
    
    def __init__(self, embedded_dim, hidden_dim, latent_dim, dropout, bias = False):
        
        super().__init__()
        
        self.attention = ScaledDotProductAttention(dropout)
        
        self.latent_norm = Normalize(latent_dim)
        
        self.norm = Normalize(embedded_dim)
        
        self.addnorm = AddNorm(embedded_dim)
        
        self.latent_dim = latent_dim
        
        self.keys_weights = nn.Linear(embedded_dim, embedded_dim, bias = bias)
        
        self.values_weights = nn.Linear(embedded_dim, embedded_dim, bias = bias)
        
        self.in_latent_linear = nn.Linear(latent_dim, embedded_dim, bias = bias)
        
        self.out_latent_linear = nn.Linear(embedded_dim, latent_dim, bias = bias)
        
        self.dense = DenseBlock(latent_dim, hidden_dim)
        
    def forward(self, latent_array, keys, values):
        
        # Layer Normalize inputs
        
        norm_latent_array = self.latent_norm(latent_array)
        
        # Pass latent array to linear layer so its dimension is the same with keys & values
        
        latent_array_1 = self.in_latent_linear(norm_latent_array)
        
        # Normalize and attach learnable parameters to keys
        
        keys = self.norm(keys)
        
        keys = self.keys_weights(keys)
        
        # Normalize and attach learnable parameters to values
        
        values = self.norm(values)
        
        values = self.values_weights(values)
        
        # Perform cross attention for latent array with keys & values
        
        cross_outputs = self.attention(latent_array_1, keys, values)
        
        # Pass the outputs to a linear layer
        
        outputs = self.out_latent_linear(cross_outputs)
        
        # Return Residual Connection
        
        res_outputs = outputs + latent_array
        
        # Pass through Dense Block
        
        #final_outputs = self.dense(res_outputs)
        
        final_outputs = res_outputs
        
        return final_outputs

## Perceiver Block

In [1365]:
class PerceiverBlock(nn.Module):
    
    def __init__(self, num_heads, embedded_dim, hidden_dim, latent_dim, 
                 self_attention_modules, cross_attention_modules, dropout):
        
        super().__init__()
        
        # self.cross_attention = CrossAttention(embedded_dim, dropout)
        
        # self.self_attention = MultiHeadAttention(embedded_dim, num_heads, dropout)
        
        self.addnorm = AddNorm(embedded_dim)
        
        self.linear_layer = nn.Linear(embedded_dim, embedded_dim)
        
        self.latent_linear = nn.Linear(latent_dim, embedded_dim)
        
        self.ffn = FFNLayer(latent_dim, hidden_dim, latent_dim)
        
        self.cross_modules = nn.Sequential()
        
        self.self_modules = nn.Sequential()
        
        for i in range(cross_attention_modules):
            self.cross_modules.add_module("cross attention"+str(i), CrossAttention(embedded_dim, hidden_dim, 
                                                                                   latent_dim, dropout))
            
        for i in range(self_attention_modules):
            self.self_modules.add_module("self attention"+str(i), MultiHeadAttention(latent_dim, hidden_dim,
                                                                                     num_heads, dropout))
        
    def forward(self, latent_inputs, inputs):
        
        for i, cross_attention in enumerate(self.cross_modules):
            
            latent_inputs = cross_attention(latent_inputs, inputs, inputs)
            
        for i, self_attention in enumerate(self.self_modules):
            
            latent_inputs = self_attention(latent_inputs, latent_inputs, latent_inputs)
              
        return latent_inputs

## Perceiver

In [1366]:
class Perceiver(nn.Module):
    
    def __init__(self, embedded_dim, hidden_dim, num_heads, num_blocks, latent_size, latent_dim, 
                 self_attention_modules, cross_attention_modules, dropout):
        
        super().__init__()
        
        self.softmax = nn.Softmax(dim=1)
        
        self.blocks = nn.Sequential()
        
        self.ffn = FFNLayer(latent_dim, hidden_dim, 2)
        
        self.latent = LatentArray(latent_size, latent_dim)
        
        self.norm = nn.LayerNorm(latent_dim)
        
        self.linear = nn.Linear(latent_dim, latent_dim)
        
        for i in range(num_blocks):
            
            self.blocks.add_module("block"+str(i), PerceiverBlock(num_heads, embedded_dim, hidden_dim, latent_dim,
                                                                  self_attention_modules, cross_attention_modules, dropout))
        
    def forward(self, inputs):
        
        # Initialize Latent Array
        
        latent_outputs = self.latent(inputs).to("cuda:0")
        
        # Running the Perceiver Blocks sequentially
        
        for i, perceiver_block in enumerate(self.blocks):
            
            latent_outputs = perceiver_block(latent_outputs, inputs)
        
        outputs = latent_outputs.mean(dim = 1)
        
        outputs_ffn = self.ffn(outputs)
        
        outputs_final = self.softmax(outputs_ffn)
              
        return outputs_final

## One Hot Encoding of the tokenised SNV data

In [1367]:
class One_Hot_Encoding(nn.Module):
    
    def __init__(self, num_classes: int):
        
        super().__init__()
        
        self.num_classes = num_classes
        
    def forward(self, input_ts):
        
        return F.one_hot(input_ts.long(), num_classes = self.num_classes)

## Positional Encoding

In [1368]:
class Positional_Encoding(nn.Module):
    
    def __init__(self, dropout):
        
        super().__init__()
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_ts):
        
        num_len = input_ts.shape[1]
        
        num_dim = input_ts.shape[2]
        
        P = torch.zeros(1, num_len, num_dim)
        
        X = (torch.arange(num_len, dtype=torch.float32).reshape(-1, 1) / 
        torch.pow(10000, torch.arange(0, num_dim, 2, dtype = torch.float32) / num_dim))
        
        P[:, :, 0::2] = torch.sin(X)
        
        if (num_dim % 2) == 1:
            
            P[:, :, 1::2] = torch.cos(X[:, :-1])
            
        else:    
            
            P[:, :, 1::2] = torch.cos(X)
        
        P = torch.repeat_interleave(P, repeats = input_ts.shape[0], dim = 0)
        
        return self.dropout(P[:, :input_ts.shape[1], :].to(input_ts.device))

## Processing the dataset before feeding into the model

In [1369]:
class prepare_dataset(nn.Module):
    
    def __init__(self, num_classes: int, phenos_or_not: bool, dropout):
        
        super().__init__()
        
        self.one_hot_encoding = One_Hot_Encoding(num_classes)
        
        self.positional_encoding = Positional_Encoding(dropout)
        
        self.num_classes = num_classes
        
        self.phenos_or_not = phenos_or_not
        
    def forward(self, snv_ts, phenos_ts):
        
        # Process tokenised SNV data
        
        encoded_snv_ts = self.one_hot_encoding(snv_ts)
        
        pos_encoded_snv_ts = self.positional_encoding(encoded_snv_ts)
        
        if self.phenos_or_not:
            
            # Process phenotypes data
            
            phenos_ts = phenos_ts.unsqueeze(0)
            
            phenos_ts = torch.repeat_interleave(phenos_ts, repeats = snv_ts.shape[1], dim = 1)
            
            phenos_ts = phenos_ts.reshape(-1, snv_ts.shape[1], phenos_ts.shape[2])
            
            # Join encoded SNV and phenotypes datasets
            
            complete_ts = torch.cat((encoded_snv_ts, pos_encoded_snv_ts, phenos_ts), dim = 2)
            
        else:
            
            complete_ts = torch.cat((encoded_snv_ts, pos_encoded_snv_ts), dim = 2)
            
            # complete_ts = snv_ts.to(torch.float32)
            
        return complete_ts
            

In [1370]:
pros = prepare_dataset(num_classes = 3, phenos_or_not = False, dropout = 0.2)

In [1371]:
train_df = pros(tr_snv, tr_phenos)

In [1372]:
tr_snv

tensor([[1, 1, 2,  ..., 0, 2, 2],
        [2, 2, 0,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 2, 1, 0],
        ...,
        [0, 2, 2,  ..., 1, 1, 2],
        [2, 2, 0,  ..., 2, 2, 1],
        [1, 0, 1,  ..., 0, 2, 1]], dtype=torch.int32)

In [1373]:
train_df.shape

torch.Size([9899, 2000, 6])

In [1374]:
test_df = pros(te_snv, te_phenos)

In [1375]:
test_df.shape

torch.Size([101, 2000, 6])

In [1376]:
tr_gout.shape

torch.Size([9899])

In [1377]:
# Pre-processing images

#image_transforms = transforms.Compose([transforms.ToTensor(),
#                                       transforms.Normalize((0.5), (0.5))])

In [1378]:
# Downloading training and testing datasets

#train_data = torchvision.datasets.MNIST(root = './mnist_data', train = True,
#                                       download = False, transform = image_transforms)

#test_data = torchvision.datasets.MNIST(root = './mnist_data', train = False,
#                                       download = False, transform = image_transforms)

In [1379]:
# Loading training and testing datasets

train_dataloader = torch.utils.data.DataLoader(train_df, batch_size = 10, shuffle = False)

label_tr_dataloader = torch.utils.data.DataLoader(tr_gout, batch_size = 10, shuffle = False)

test_dataloader = torch.utils.data.DataLoader(test_df, batch_size = 4, shuffle = False)

label_te_dataloader = torch.utils.data.DataLoader(te_gout, batch_size = 4, shuffle = False)

In [1392]:
# Loading training and testing datasets

# train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

In [1481]:
#model = Perceiver(embedded_dim = 2000, num_heads = 1, hidden_dim = 32, num_blocks = 1, latent_size = 4, latent_dim = 6,
#                  self_attention_modules = 1, cross_attention_modules = 1, dropout = 0.5)

In [1482]:
model = Perceiver(embedded_dim = 6, num_heads = 1, hidden_dim = 1, num_blocks = 1, latent_size = 6, latent_dim = 4,
                  self_attention_modules = 1, cross_attention_modules = 1, dropout = 0.5)

In [1483]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 5e-7, amsgrad = True)

epochs = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1484]:
model = model.to(device)

In [1485]:
n_total_steps = len(train_dataloader)

In [1486]:
start_time = time.time()

In [1487]:
model.train()

for epoch in range(epochs):
    
    for i, (images, labels) in enumerate(zip(train_dataloader, label_tr_dataloader)):
        
        # labels = torch.unsqueeze(labels, 1)
        
        #images = torch.unsqueeze(images, 1)
        
        images = images.to(device)
        
        labels = labels.to(device)
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        if (i+1) % 200 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.3f}')

Epoch [1/2], Step [200/990], Loss: 0.912
Epoch [1/2], Step [400/990], Loss: 0.769
Epoch [1/2], Step [600/990], Loss: 0.721
Epoch [1/2], Step [800/990], Loss: 0.626
Epoch [2/2], Step [200/990], Loss: 0.912
Epoch [2/2], Step [400/990], Loss: 0.769
Epoch [2/2], Step [600/990], Loss: 0.721
Epoch [2/2], Step [800/990], Loss: 0.626


In [1488]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 36.452054262161255 seconds ---


In [1489]:
# List of labels

classes = ('0', '1')

In [1490]:
model.eval()

# Evaluate the trained model performance

with torch.no_grad():
    
    batch_size = 1
    
    num_correct_preds = 0
    
    num_total = len(test_df)
    
    num_correct_per_label = [0] * len(classes)
    
    num_total_per_label = [0] * len(classes)
    
    for i, (images, labels) in enumerate(zip(test_dataloader, label_te_dataloader)):
        
        #images = torch.unsqueeze(images, 1)
        
        images = images.to(device)
        
        labels = labels.to(device)
        
        outputs = model(images)
        
        # Return the value with the highest probability score
        
        _, pred_values = torch.max(outputs, 1)
        
        num_correct_preds += (pred_values == labels).sum().item()        
    
        for i in range(batch_size):
            
            label = labels[i]
            
            pred_val = pred_values[i]
            
            num_total_per_label[label] += 1 
            
            if label == pred_val:
                
                num_correct_per_label[label] += 1
                
    # Calculate Overall Accuracy
    
    overall_accuracy = 100.0 * num_correct_preds / num_total
    
    print(f'Overall accuracy of the model: {overall_accuracy:.3f} %')
    
    # Calculate Accuracy per Label
    
    for i in range(len(classes)):
        
        accuracy_per_label = 100.0 * num_correct_per_label[i] / num_total_per_label[i]
        
        print(f'Accuracy of Label {classes[i]} : {accuracy_per_label:.3f} %')

Overall accuracy of the model: 58.416 %
Accuracy of Label 0 : 100.000 %
Accuracy of Label 1 : 0.000 %
